In [109]:
import sympy as sp
import numpy as np

import re
from sympy import symbols, Symbol, init_printing

# 1) Symbol を継承して _latex をオーバーライド
class DoubleIndexSymbol(Symbol):
    def _latex(self, printer):
        m = re.fullmatch(r'([A-Za-z]+)_(\d+)_(\d+)__(\d+)', self.name)
        if m:
            base, i, j, k = m.groups()
            return f"{base}_{{{i},{j}}}^{k}"
        return self.name

# 2) MathJax 表示を有効化
init_printing(use_latex='mathjax')

In [110]:
file_path = 'data/n3-d3.dat-s'

n = 3
d = 3

In [111]:
def decode_variable_index(var_idx):
    if var_idx == 0:
        return 'x_0'
    var_idx -= 1
    i = var_idx % (n + 1)
    var_idx //= (n + 1)
    j = var_idx % (n + 1)
    var_idx //= (n + 1)
    t = var_idx % (n + 1)
    return f"x_{i}_{j}__{t}"

In [112]:
with open(file_path) as f:
    line = f.readline()
    num_variables = int(line.split()[0])

    variables = sp.symbols(" ".join([decode_variable_index(i) for i in range(num_variables + 1)]), cls=DoubleIndexSymbol)

    line = f.readline()
    nblock = int(line.split()[0])

    line = f.readline()
    block_sizes = map(int, line.split())

    line = f.readline()
    objective = map(int, line.split())
    
    line = f.readline()

    matrices = [
        sp.Matrix(abs(s), abs(s), lambda i,j: 0) for s in block_sizes
    ]

    while line:
        var_idx, block_idx, row, col, val = map(int, line.split())
        if var_idx == 0:
            matrices[block_idx - 1][row - 1, col - 1] += (-1) * val
        else:
            matrices[block_idx - 1][row - 1, col - 1] += val * variables[var_idx]
        line = f.readline()

In [113]:
# symmetrize
for mat in matrices:
    rows, cols = mat.shape
    for row in range(rows):
        for col in range(cols):
            if row <= col:
                continue
            mat[row, col] = mat[col, row]

In [114]:
vars = sp.symbols("x_0_0__0 x_1_0__0")
vars[0], vars[1]

(x⁰₀ ₀, x⁰₁ ₀)

In [115]:
print(variables)

(x_0, x_0_0__0, x_1_0__0, x_2_0__0, x_3_0__0, x_0_1__0, x_1_1__0, x_2_1__0, x_3_1__0, x_0_2__0, x_1_2__0, x_2_2__0, x_3_2__0, x_0_3__0, x_1_3__0, x_2_3__0, x_3_3__0, x_0_0__1, x_1_0__1, x_2_0__1, x_3_0__1, x_0_1__1, x_1_1__1, x_2_1__1, x_3_1__1, x_0_2__1, x_1_2__1, x_2_2__1, x_3_2__1, x_0_3__1, x_1_3__1, x_2_3__1, x_3_3__1, x_0_0__2, x_1_0__2, x_2_0__2, x_3_0__2, x_0_1__2, x_1_1__2, x_2_1__2, x_3_1__2, x_0_2__2, x_1_2__2, x_2_2__2, x_3_2__2, x_0_3__2, x_1_3__2, x_2_3__2, x_3_3__2, x_0_0__3, x_1_0__3, x_2_0__3, x_3_0__3, x_0_1__3, x_1_1__3, x_2_1__3, x_3_1__3, x_0_2__3, x_1_2__3, x_2_2__3, x_3_2__3, x_0_3__3, x_1_3__3, x_2_3__3, x_3_3__3)


In [116]:
block_idx = 1

matrices[block_idx - 1]

⎡ x⁰₀ ₀        3⋅x⁰₁ ₀            3⋅x⁰₂ ₀        x⁰₃ ₀ ⎤
⎢                                                      ⎥
⎢3⋅x⁰₁ ₀  3⋅x⁰₀ ₀ + 6⋅x⁰₂ ₀  6⋅x⁰₁ ₀ + 3⋅x⁰₃ ₀  3⋅x⁰₂ ₀⎥
⎢                                                      ⎥
⎢3⋅x⁰₂ ₀  6⋅x⁰₁ ₀ + 3⋅x⁰₃ ₀  3⋅x⁰₀ ₀ + 6⋅x⁰₂ ₀  3⋅x⁰₁ ₀⎥
⎢                                                      ⎥
⎣ x⁰₃ ₀        3⋅x⁰₂ ₀            3⋅x⁰₁ ₀        x³₃ ₃ ⎦

In [117]:
block_idx = 100

matrices[block_idx - 1]

[-2⋅x⁰₀ ₀ + x²₀ ₀ + 1]

In [118]:
selected_vars = set([1, 4, 13, 64])

subs_map = {}
for i in range(1, num_variables + 1):
    if i in selected_vars:
        subs_map[variables[i]] = 1
    else:
        subs_map[variables[i]] = 0

print(subs_map)

subs_matrices = []
for mat in matrices:
    subs_matrices.append(mat.subs(subs_map))

subs_matrices[0]

{x_0_0__0: 1, x_1_0__0: 0, x_2_0__0: 0, x_3_0__0: 1, x_0_1__0: 0, x_1_1__0: 0, x_2_1__0: 0, x_3_1__0: 0, x_0_2__0: 0, x_1_2__0: 0, x_2_2__0: 0, x_3_2__0: 0, x_0_3__0: 1, x_1_3__0: 0, x_2_3__0: 0, x_3_3__0: 0, x_0_0__1: 0, x_1_0__1: 0, x_2_0__1: 0, x_3_0__1: 0, x_0_1__1: 0, x_1_1__1: 0, x_2_1__1: 0, x_3_1__1: 0, x_0_2__1: 0, x_1_2__1: 0, x_2_2__1: 0, x_3_2__1: 0, x_0_3__1: 0, x_1_3__1: 0, x_2_3__1: 0, x_3_3__1: 0, x_0_0__2: 0, x_1_0__2: 0, x_2_0__2: 0, x_3_0__2: 0, x_0_1__2: 0, x_1_1__2: 0, x_2_1__2: 0, x_3_1__2: 0, x_0_2__2: 0, x_1_2__2: 0, x_2_2__2: 0, x_3_2__2: 0, x_0_3__2: 0, x_1_3__2: 0, x_2_3__2: 0, x_3_3__2: 0, x_0_0__3: 0, x_1_0__3: 0, x_2_0__3: 0, x_3_0__3: 0, x_0_1__3: 0, x_1_1__3: 0, x_2_1__3: 0, x_3_1__3: 0, x_0_2__3: 0, x_1_2__3: 0, x_2_2__3: 0, x_3_2__3: 0, x_0_3__3: 0, x_1_3__3: 0, x_2_3__3: 0, x_3_3__3: 1}


⎡1  0  0  1⎤
⎢          ⎥
⎢0  3  3  0⎥
⎢          ⎥
⎢0  3  3  0⎥
⎢          ⎥
⎣1  0  0  1⎦

In [119]:
for block_idx, mat in enumerate(subs_matrices):
    eigenvals = mat.eigenvals()
    is_psd = all(ev >= 0 for ev in eigenvals.keys())

    if not is_psd:
        print(f'block [{block_idx + 1}] is not positive semedefinite!')
        display(mat)
        display(matrices[block_idx])

block [83] is not positive semedefinite!


[-1]

[-x⁰₀ ₃ - x⁰₃ ₀ + x⁰₃ ₃ + 1]

block [84] is not positive semedefinite!


[-1]

...

[-1]

[-x⁰₀ ₀ - x⁰₀ ₃ + x³₀ ₃ + 1]

block [128] is not positive semedefinite!


[-1]

[-x⁰₀ ₀ - x⁰₃ ₀ + x³₃ ₀ + 1]